In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import nltk
import nltk.corpus

In [ ]:
tweet_data=pd.read_csv("../input/nlp-getting-started/train.csv")
tweet_data=tweet_data.drop(["keyword","location"],axis=1)
tweet_data

In [ ]:
test_data=pd.read_csv("../input/nlp-getting-started/test.csv")
test_data=test_data.drop(["keyword","location"],axis=1)
test_data

In [ ]:
data=pd.DataFrame(tweet_data.text,columns=["text"])
data2=pd.DataFrame(test_data.text,columns=["text"])
Data=pd.concat([data,data2],axis=0,ignore_index=True)

In [ ]:
import re
import string

def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)

def remove_html(text):
    html = re.compile(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    return re.sub(html, '', text)

# Applying helper functions
Data['text'] = Data['text'].apply(lambda x: remove_URL(x))
Data['text'] = Data['text'].apply(lambda x: remove_html(x))

In [ ]:
for i in range(0,10876):
    Data["text"][i]=Data["text"][i].lower()

In [ ]:
from nltk.corpus import stopwords
stop_words=set(stopwords.words("english"))
print(stopwords.words("english"))
stop_wrds=np.array(stop_words)

In [ ]:
import gensim
Data["tokenized_text"]=Data["text"].apply(gensim.utils.simple_preprocess)
Data

In [ ]:
#text = ' '.join([word for word in txt.split() if word not in stopwords.words('english')])

In [ ]:
#filtered_sentence = []
#
#if Data["text"][0] not in stop_words:
#    filtered_sentence.append(Data["text"][0])

In [ ]:
def remove_stp (txt_tokenized):
    txt_clean=[word for word in txt_tokenized if word not in stop_words]
    return txt_clean
Data['no_stpword'] = Data['tokenized_text'].apply(lambda x: remove_stp(x))


In [ ]:
Data

In [ ]:
final=[0]*10876
for i in range(0,10876):
    str=" "
    re=Data["no_stpword"][i]
    final[i]=str.join(re)

In [ ]:
Data["final_txt"]=final             #pd.DataFrame([final],columns=["final_txt"])
Data

In [ ]:
#filtered_sentence = []
 
#for w in Data["tokenized_text"]:
#    if w not in stop_wrds:
#        filtered_sentence.append(w)

In [ ]:
Data.text

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()
vector=cv.fit_transform(Data["final_txt"])

In [ ]:
#X=vector[0:7612]

In [ ]:
X=vector[0:7613]
y=tweet_data.target
#X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.90)

In [ ]:
rf=MultinomialNB()
#rf=RandomForestClassifier(n_estimators=65)
#rf.fit(X_train,y_train)
#pred=rf.predict(X_test)

In [ ]:
#accuracy_score(pred,y_test)

In [ ]:
#rf=RandomForestClassifier(n_estimators=85,bootstrap=True)
rf.fit(X,y)
pred=rf.predict(X)
accuracy_score(pred,y)

In [ ]:
test_vector=vector[7613:10877]

In [ ]:
pre=rf.predict(test_vector)
datframe=pd.DataFrame(pre,columns=["target"])
datframe

In [ ]:
sns.countplot(x="target",data=datframe)

In [ ]:
sam=pd.read_csv("../input/nlp-getting-started/sample_submission.csv")
sam

In [ ]:
sub=sam.drop(["target"],axis=1)

In [ ]:
submission=pd.concat([sub,datframe],axis=1)
submission

In [ ]:
submission.to_csv("submission.csv",index=False)

In [ ]:
!kaggle competitions submit -c nlp-getting-started -f submission.csv -m "Message"